# OpenML Regression Benchmark 2025 – **Complete Notebook**

This notebook benchmarks a variety of regression models on the `New_OpenML_Suite_2025_regression` collection, logs experiments to **Weights & Biases**, runs hyper‑parameter sweeps, and tracks full data/model lineage.


In [61]:
%pip install -Uq pip openml scikit-learn pandas numpy nbformat setuptools

Note: you may need to restart the kernel to use updated packages.


In [ ]:
# Setup
import numpy as np
from IPython.display import display

In [2]:
# Configuration
import openml

SUITE_ID = "8f0ea660163b436bbd4abd49665c7b1d"  # OpenML-CTR23 - A curated tabular regression benchmarking suite
suite = openml.study.get_suite(SUITE_ID)
display(suite)

OpenML Benchmark Suite
ID..............: 353
Name............: OpenML-CTR23 - A curated tabular regression benchmarking suite
Status..........: active
Main Entity Type: task
Study URL.......: https://www.openml.org/s/353
# of Data.......: 35
# of Tasks......: 35
Creator.........: https://www.openml.org/u/30127
Upload Time.....: 2023-05-31 16:39:49

In [3]:
# Data Preprocessing
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler


def make_preprocessor(dataset: openml.OpenMLDataset) -> ColumnTransformer:
    target = dataset.default_target_attribute
    categorical_indicator, attribute_names = dataset.get_data(
        dataset_format="dataframe"
    )[2:4]
    np_cat_ind = np.array(categorical_indicator)
    np_attr_name = np.array(attribute_names)
    feature_mask = np.where(np_attr_name != target)
    np_cat_ind = np_cat_ind[feature_mask]
    np_attr_name = np_attr_name[feature_mask]

    numeric_features = np_attr_name[~np_cat_ind].tolist()
    nominal_features = np_attr_name[np_cat_ind].tolist()

    numeric_pipeline = Pipeline(steps=[("scaler", StandardScaler())])
    nominal_pipeline = Pipeline(
        steps=[("onehot", OneHotEncoder(handle_unknown="ignore", max_categories=20, sparse_output=False))]
    )

    preprocessor = ColumnTransformer(
        transformers=[
            ("numeric", numeric_pipeline, numeric_features),
            ("nominal", nominal_pipeline, nominal_features),
        ],
        sparse_threshold=0.0,
    )
    return preprocessor

In [4]:
# Baseline Models
from sklearn.dummy import DummyRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.neural_network import MLPRegressor

# Baseline models
models = {
    "DummyRegressor": DummyRegressor(),
    "RandomForestRegressor": RandomForestRegressor(),
    "LinearRegression": LinearRegression(),
    "MLPRegressor": MLPRegressor(),
}

In [5]:
# Hyperparameter Optimization

search_params = {
    "cv": 5,
    "n_jobs": -1,
    "n_trials": 4,
    "random_state": 42,
    "return_train_score": False,
    # "scoring": "neg_mean_squared_error",
    "timeout": None,
    "verbose": 0,
}

hpo_grid = {
    "RandomForestRegressor": {
        "n_estimators": [5, 10],
        "max_depth": [2, 5],
    },
    "MLPRegressor": {
        "hidden_layer_sizes": [(5,), (10,)],
        "learning_rate_init": [1e-3, 1e-1],
        "max_iter": [100, 200],
    },
}

In [6]:
from sklearn.model_selection import GridSearchCV, KFold


def make_hpo_model(preprocessor: ColumnTransformer, model_name: str):
    pipe = Pipeline(
        steps=[("preprocessor", preprocessor), ("model", models[model_name])]
    )
    return (
        GridSearchCV(
            pipe,
            {f"model__{k}": v for k, v in hpo_grid[model_name].items()},
            cv=KFold(n_splits=4, shuffle=True),
        )
        if model_name in hpo_grid
        else pipe
    )

In [ ]:
def evaluate_model(task: openml.OpenMLTask, model_name: str, seed: int = 42):
    dataset = task.get_dataset()
    display(f"Running {model_name} on task {task}...")
    preprocessor = make_preprocessor(dataset)
    model = make_hpo_model(preprocessor, model_name)
    run = openml.runs.run_model_on_task(model=model, task=task, seed=seed, n_jobs=-1)
    return run

In [ ]:
# Run Models
N_TASKS = 5

for task_id in suite.tasks[:N_TASKS]:
    task = openml.tasks.get_task(task_id)
    for model_name in models.keys():
        run = evaluate_model(task, model_name)
        display(run)
        break
    break

OpenML Regression Task
Task Type Description: https://www.openml.org/tt/TaskType.SUPERVISED_REGRESSION
Task ID..............: 361234
Task URL.............: https://www.openml.org/t/361234
Estimation Procedure.: crossvalidation
Target Feature.......: rings

OpenML Dataset
Name.........: abalone
Version......: 15
Format.......: arff
Upload Date..: 2022-12-22 16:10:55
Licence......: CC BY 4.0
Download URL.: https://api.openml.org/data/v1/download/22111820/abalone.arff
OpenML URL...: https://www.openml.org/d/44956
# of features: None

Running DummyRegressor on task 361234...


OpenML Run
Uploader Name..............: None
Metric.....................: None
Local Result - MAE (+- STD): 2.3628 +- 0.1077
Local Runtime - ms (+- STD): 19.2097 +- 1.9058
Run ID.....................: None
Task ID....................: 361234
Task Type..................: None
Task URL...................: https://www.openml.org/t/361234
Flow ID....................: None
Flow Name..................: sklearn.pipeline.Pipeline(preprocessor=sklearn.compose._column_transformer.ColumnTransformer(numeric=sklearn.pipeline.Pipeline(scaler=sklearn.preprocessing._data.StandardScaler),nominal=sklearn.pipeline.Pipeline(onehot=sklearn.preprocessing._encoders.OneHotEncoder)),model=sklearn.dummy.DummyRegressor)
Flow URL...................: None
Setup ID...................: None
Setup String...............: Python_3.13.3. Sklearn_1.6.1. NumPy_2.2.5. SciPy_1.15.2.
Dataset ID.................: 44956
Dataset URL................: https://www.openml.org/d/44956

Running RandomForestRegressor on task 361234...


...

Running LinearRegression on task 361241...


OpenML Run
Uploader Name..............: None
Metric.....................: None
Local Result - MAE (+- STD): 4.3414 +- 0.0388
Local Runtime - ms (+- STD): 32.5022 +- 4.0689
Run ID.....................: None
Task ID....................: 361241
Task Type..................: None
Task URL...................: https://www.openml.org/t/361241
Flow ID....................: None
Flow Name..................: sklearn.pipeline.Pipeline(preprocessor=sklearn.compose._column_transformer.ColumnTransformer(numeric=sklearn.pipeline.Pipeline(scaler=sklearn.preprocessing._data.StandardScaler),nominal=sklearn.pipeline.Pipeline(onehot=sklearn.preprocessing._encoders.OneHotEncoder)),model=sklearn.linear_model._base.LinearRegression)
Flow URL...................: None
Setup ID...................: None
Setup String...............: Python_3.13.3. Sklearn_1.6.1. NumPy_2.2.5. SciPy_1.15.2.
Dataset ID.................: 44963
Dataset URL................: https://www.openml.org/d/44963

Running MLPRegressor on task 361241...


/home/ahteh/Developer/research/.venv/lib/python3.13/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/ahteh/Developer/research/.venv/lib/python3.13/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/ahteh/Developer/research/.venv/lib/python3.13/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/ahteh/Developer/research/.venv/lib/python3.13/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged ye

OpenML Run
Uploader Name..............: None
Metric.....................: None
Local Result - MAE (+- STD): 3.8078 +- 0.0721
Local Runtime - ms (+- STD): 203850.5184 +- 7509.2122
Run ID.....................: None
Task ID....................: 361241
Task Type..................: None
Task URL...................: https://www.openml.org/t/361241
Flow ID....................: None
Flow Name..................: sklearn.model_selection._search.GridSearchCV(estimator=sklearn.pipeline.Pipeline(preprocessor=sklearn.compose._column_transformer.ColumnTransformer(numeric=sklearn.pipeline.Pipeline(scaler=sklearn.preprocessing._data.StandardScaler),nominal=sklearn.pipeline.Pipeline(onehot=sklearn.preprocessing._encoders.OneHotEncoder)),model=sklearn.neural_network._multilayer_perceptron.MLPRegressor))
Flow URL...................: None
Setup ID...................: None
Setup String...............: Python_3.13.3. Sklearn_1.6.1. NumPy_2.2.5. SciPy_1.15.2.
Dataset ID.................: 44963
Dataset URL....